<a href="https://colab.research.google.com/github/massinoLight/tp_AlgoBD/blob/main/TP_Regression_lineaire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install pyspark

##On charge le jeu de données

In [5]:
import requests 
file_url = "https://raw.githubusercontent.com/apache/spark/master/data/mllib/ridge-data/lpsa.data"
	
r = requests.get(file_url, stream = True) 

with open("/content/sample_data/lpsa.data", "wb") as file: 
	for block in r.iter_content(chunk_size = 1024): 
		if block: 
			file.write(block) 

##Spark context

In [6]:
from pyspark import SparkContext
# $example on$
from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD, LinearRegressionModel

sc = SparkContext(appName="PythonLinearRegressionWithSGDExample")

##Fonction pour préparer le jeux de données

In [7]:
def parsePoint(line):
        values = [float(x) for x in line.replace(',', ' ').split(' ')]
        return LabeledPoint(values[0], values[1:])

##On applique le modéle a notre jeux de données que prépare au fur et a mesure (nb iterations=100)

In [8]:
data = sc.textFile("/content/sample_data/lpsa.data")
parsedData = data.map(parsePoint)

model = LinearRegressionWithSGD.train(parsedData, iterations=100, step=0.00000001)

valuesAndPreds = parsedData.map(lambda p: (p.label, model.predict(p.features)))
MSE = valuesAndPreds \
        .map(lambda vp: (vp[0] - vp[1])**2) \
        .reduce(lambda x, y: x + y) / valuesAndPreds.count()
print("Mean Squared Error = " + str(MSE))

    

/usr/local/lib/python3.7/dist-packages/pyspark/mllib/regression.py:302: FutureWarning: Deprecated in 2.0.0. Use ml.regression.LinearRegression.
  "Deprecated in 2.0.0. Use ml.regression.LinearRegression.", FutureWarning)


Mean Squared Error = 7.4510328101026015


In [9]:
# Save and load model
model.save(sc, "model/LR/pythonLinearRegressionWithSGDModel")
sameModel = LinearRegressionModel.load(sc, "model/LR/pythonLinearRegressionWithSGDModel")